In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils import to_categorical
from unidecode import unidecode
import numpy as np
import random, sys, io, re, csv

Using TensorFlow backend.


In [2]:
csv_reader = csv.reader(open('data\\lyrics.csv', encoding='utf-8'))

In [3]:
# return a dictionary of song: lyrics
def get_tokenized_lines(csv):
    lyrics = {}
    for r in csv:
        words = []
        row = str(r[2]).lower()
        for line in row.split('|-|'):
            new_words = re.findall(r"\b[a-z']+\b", unidecode(line))
            words = words + new_words
        lyrics[r[1]] = words
    return lyrics

In [4]:
all_lyric_words = get_tokenized_lines(csv_reader)

In [5]:
# total word number: 173631, total lines of lyrics: 22688, average word per line: 7.652988
SEQ_LENGTH = 23
sequences = list()

def get_all_sequences():
    for song in all_lyric_words:
        if len(all_lyric_words[song]) < SEQ_LENGTH:
            sequences.append(all_lyric_words[song])
        else:
            for i in range(SEQ_LENGTH, len(all_lyric_words[song])):
                seq = all_lyric_words[song][i - SEQ_LENGTH: i]
                sequences.append(seq)
    
    return sequences

In [6]:
sequences = get_all_sequences()
print('Total Sequences: %d' % len(sequences))

Total Sequences: 166036


In [7]:
# store all the unique words and match them with indices
all_words = []
for song in all_lyric_words:
    for word in all_lyric_words[song]:
        all_words.append(word)
unique_word = set(all_words)
word_to_index = {w: i for i, w in enumerate(unique_word)}
index_to_word = {i: w for w, i in word_to_index.items()}
word_indices = [word_to_index[word] for word in unique_word]
word_size = len(unique_word)

print('vocabulary size: {}'.format(word_size))

vocabulary size: 9681


In [8]:
# this funtion change the words into matrix wise data, and each position of the matrix stands for the index of the word in index_to_word

def data_to_matrix(lines, seq_len):
    matrix = np.zeros((len(lines), seq_len))
    
    for r, line in enumerate(lines):
        for c, word in enumerate(line):
            matrix[r, c] = word_to_index[word]

    return matrix

In [9]:
matrix_data = data_to_matrix(sequences, SEQ_LENGTH)

NameError: name 'tokenized' is not defined

In [13]:
matrix_data[:,-1].shape

(166375,)

In [14]:
# get the y = Wx data, X
X, y = matrix_data[:, :], matrix_data[:, -1]
y = to_categorical(y, num_classes=word_size)
seq_length = len(X[0])

In [15]:
print("X_shape", X.shape)
print("y_shape", y.shape)

X_shape (166375, 21)
y_shape (166375, 9681)
